In [1]:
from openai import OpenAI
import json
import chromadb
import pandas as pd 
from AI_Functions import call_llm
from RAG_Functions import create_vectors, Search_listings, get_propery_description
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

# create an OpenAI client
client = OpenAI()

#create a chromaDB client
chroma_client = chromadb.PersistentClient()
Real_Estate = chroma_client.get_or_create_collection(name="Real_Estate" , metadata={"hnsw:space": "cosine"} )

# Check if there is any existing data in the collection
Real_Estate.count()


40

In [2]:
questions = [   
                "How big do you want your house to be?" ,
                "What are 3 most important things for you in choosing a property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]

In [3]:
answers = []
for i, question in enumerate(questions):
    while True:
        answer = input(question + "\n").strip()  # .strip() removes leading/trailing whitespace
        if answer:  # Checks if the answer is not empty
            answers.append(answer)
            break  # Exit the while loop and proceed to the next question
        else:
            print("Please provide an answer.")


In [4]:
print("Thank you for your answers. Here are your answers:"+ str(answers))

Thank you for your answers. Here are your answers:['4 Bed Room', 'Parks, Commute and schools', 'Big Garden', 'Train and Car', 'Beach Side']


In [5]:

#Defind an empty dataframe with columns
df = pd.DataFrame(columns=['Property_ID', 'Answer1_Similarities', 'Answer2_Similarities', 'Answer3_Similarities', 'Answer4_Similarities', 'Answer5_Similarities'])

In [6]:
default_values = {
    'Property_ID': None,  # I will provide it later
    'Answer1_Similarities': 2,
    'Answer2_Similarities': 2,
    'Answer3_Similarities': 2,
    'Answer4_Similarities': 2,
    'Answer5_Similarities': 2  # Corrected to include Answer5_Similarities
}
column_names = ['Answer1_Similarities', 'Answer2_Similarities', 'Answer3_Similarities', 'Answer4_Similarities', 'Answer5_Similarities']

for answer_id,answer in enumerate(answers):
    print("Searching for listings similar to: " + answer)
    answer_vector = create_vectors(answer, EMBEDDING_MODEL_NAME)
    Listings = Search_listings(Real_Estate, answer_vector,3)
    #print(Listings)
    # Creating a subset of Listings, that match the customer preferences, along with the closeness of the match per answer
    for i in range(len(Listings['ids'][0])):
        listing = Real_Estate.get(ids=[Listings['ids'][0][i]])
        property_ID = listing['metadatas'][0]['property_ID']
        if property_ID not in df['Property_ID'].values:  # add the listing to the dataframe only if it is not already in the dataframe
            row_values = default_values.copy()
            row_values['Property_ID'] = property_ID
            row_df = pd.DataFrame([row_values])
            df = pd.concat([df, row_df], ignore_index=True)
        df.loc[df['Property_ID'] == property_ID, column_names[answer_id]] = Listings['distances'][0][i] # Collect the similarity score for the listing per answer

Searching for listings similar to: 4 Bed Room
Searching for listings similar to: Parks, Commute and schools
Searching for listings similar to: Big Garden
Searching for listings similar to: Train and Car
Searching for listings similar to: Beach Side


In [7]:
# Next Step is to sum the columns to get a total score for each listing. This will help to match the listings to the customer preferences in order of relevance
df['Total_Score'] = df[column_names].sum(axis=1)

In [8]:
df

,Property_ID,Answer1_Similarities,Answer2_Similarities,Answer3_Similarities,Answer4_Similarities,Answer5_Similarities,Total_Score
0,6757e520-b442-4305-9cc8-f90b90110832,0.142659,2,0.174728,2,2,6.317388
1,2393d7c0-382d-4df6-950a-396dc6c7debd,0.144527,2,2,2,2,8.144527
2,3c4e5037-8a23-4bfd-ad48-7ad152631565,0.153842,0.192189,0.157725,2,2,4.503756
3,25f57fd5-2dc8-47a4-9b30-07e0d1571c0f,2,0.188777,2,2,2,8.188777
4,0e2845f4-33e9-4195-be0f-7b2968811344,2,0.191757,2,2,2,8.191757
5,51b9fafe-8961-4f6e-9e26-39d1d7052e62,2,2,0.174824,0.252998,2,6.427822
6,0fecae02-04ed-462d-9617-6195e7f8de00,2,2,2,0.250843,2,8.250843
7,75a06c47-7b53-4c16-8d24-684a5cd8cf50,2,2,2,2,0.1631,8.1631
8,22c3a7da-f33e-414c-9256-0bc6d792f599,2,2,2,2,0.180647,8.180647


In [9]:
#Now, Lets take the top 3 listings that match the customer preferences
df['Total_Score'] = pd.to_numeric(df['Total_Score'], errors='coerce')
top_3_listings = df.nsmallest(3, 'Total_Score')

In [10]:
top_3_listings

,Property_ID,Answer1_Similarities,Answer2_Similarities,Answer3_Similarities,Answer4_Similarities,Answer5_Similarities,Total_Score
2,3c4e5037-8a23-4bfd-ad48-7ad152631565,0.153842,0.192189,0.157725,2,2,4.503756
0,6757e520-b442-4305-9cc8-f90b90110832,0.142659,2,0.174728,2,2,6.317388
5,51b9fafe-8961-4f6e-9e26-39d1d7052e62,2,2,0.174824,0.252998,2,6.427822


In [11]:
print(f"Here are the Top 3 houses that match your preferences:")

for property in top_3_listings['Property_ID']:
    print(f"Property ID: {property}")
    Description = get_propery_description(Real_Estate, property,"Description")
    Neighborhood_Description = get_propery_description(Real_Estate, property,"Neighborhood_Description")
    print(Description['documents'][0] + "\n" + Neighborhood_Description['documents'][0])
    print("\n")






Here are the Top 3 houses that match your preferences:
Property ID: 3c4e5037-8a23-4bfd-ad48-7ad152631565
Elegant 4-bedroom Victorian home with a garden oasis. This grand residence features period details, a gourmet kitchen, and a tranquil outdoor space for relaxation.
Albert Park is a prestigious suburb with leafy streets, upscale dining options, and close proximity to Albert Park Lake.


Property ID: 6757e520-b442-4305-9cc8-f90b90110832
Spacious 4-bedroom family home with a landscaped backyard. This renovated property features a gourmet kitchen, multiple living areas, and easy access to parks and schools.
Fitzroy North is a leafy suburb with a strong community vibe, local cafes, and recreational facilities.


Property ID: 51b9fafe-8961-4f6e-9e26-39d1d7052e62
Charming 3-bedroom Victorian home with a beautifully landscaped garden. This heritage property exudes character and elegance, with original stained glass windows and period features throughout.
South Yarra is a vibrant neighborhoo

In [20]:
for i,column in enumerate(column_names):
    df[column] = pd.to_numeric(df[column], errors='coerce')
    top_3_listings_per_answer = df.nsmallest(3, column)
    #print(top_3_listings_per_answer)
    print(f"Here are the Top 3 houses that match your preferences for {answers[i]}:\n")
    for property in top_3_listings_per_answer['Property_ID']:
        Description = get_propery_description(Real_Estate, property,"Description")
        Neighborhood_Description = get_propery_description(Real_Estate, property,"Neighborhood_Description")
        Base_Description = Description['documents'][0] + "\n" + Neighborhood_Description['documents'][0]
        messages = [{"role": "user", "content": f"create a personalized real estate listing for this  Base Description: {Base_Description} , Based on the Personal Preference of {answers[i]}"}] # Creating one by one is leading to repeated listings, so creating 10 at a time. Increase the temperature as well
        response_message = call_llm("gpt-3.5-turbo", messages,tools=None,temperature=0.9)
        print(response_message.content)
        print("\n")


Here are the Top 3 houses that match your preferences for 4 Bed Room:

Welcome to your dream family home in Fitzroy North! This spacious 4-bedroom property is perfect for a growing family looking for a comfortable and stylish living space.

As you enter the home, you'll be greeted by a beautifully landscaped backyard, perfect for relaxing or entertaining guests. The interior of the home has been recently renovated to feature a gourmet kitchen, complete with high-end appliances and ample storage space. The open-plan layout includes multiple living areas, allowing for plenty of space for family time or hosting gatherings.

Located in the leafy suburb of Fitzroy North, this home offers easy access to nearby parks and schools, making it ideal for families with young children. The local community vibe is strong, with plenty of cafes and recreational facilities to explore.

Don't miss the opportunity to make this family-friendly home yours. Schedule a viewing today and start envisioning your